# electricity_forecast.API.ipynb
Tiny demo showing the helper API. Runs on a small slice of your CSV.

# Keras Tuner API Demonstration

This notebook demonstrates the **native Keras Tuner API** and a small
**wrapper layer** implemented in this project.

The goal of this notebook is to show **how the tool works**, not to
solve a specific forecasting problem. All project-specific logic
and experiments are documented separately in the example notebooks.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

## Defining a HyperModel

Keras Tuner searches over model configurations defined through a
model-building function. This function takes a `HyperParameters`
object and returns a compiled Keras model.

In [4]:
def build_dense_model(hp):
    units = hp.Int("units", min_value=16, max_value=64, step=16)
    lr = hp.Choice("learning_rate", [1e-2, 1e-3])

    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(1)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mse"
    )

    return model

In [5]:
# synthetic data for API demonstration
X = np.random.randn(300, 10)
y = np.random.randn(300, 1)

## Running a Minimal Hyperparameter Search

A tuner evaluates multiple model configurations and tracks their
validation performance. Here, a small random search is used for
demonstration purposes.

In [7]:
tuner = kt.RandomSearch(
    build_dense_model,
    objective="val_loss",
    max_trials=2,
    directory="api_demo",
    project_name="random_search_example"
)

tuner.search(
    X, y,
    validation_split=0.2,
    epochs=3,
    verbose=1
)

Trial 2 Complete [00h 00m 01s]
val_loss: 0.9634771943092346

Best val_loss So Far: 0.9129384160041809
Total elapsed time: 00h 00m 01s


In [8]:
best_model = tuner.get_best_models(1)[0]
best_model.summary()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769 (3.00 KB)

 Trainable params: 769 (3.00 KB)

 Non-trainable params: 0 (0.00 B)

## Lightweight Wrapper Layer

This project includes a small utility module that wraps common Keras
operations. The wrapper does not change Keras Tuner behavior; it only
simplifies repetitive tasks and keeps notebooks clean.

In [9]:
from electricity_forecast_utils import build_lstm_model

model = build_lstm_model(input_shape=(10, 1))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,961 (66.25 KB)

 Trainable params: 16,961 (66.25 KB)

 Non-trainable params: 0 (0.00 B)

## Summary

This notebook demonstrated:
- the native Keras Tuner API
- how to define a search space
- how to run a minimal hyperparameter search
- how the project’s wrapper layer integrates with Keras

All project-specific modeling and evaluation are covered in the
example notebooks.